In [42]:
import requests
import json
from gtts import gTTS
import tempfile
from pygame import mixer
import transform_stationname2

def say(text, filename=None):
    with tempfile.NamedTemporaryFile(delete=True) as temp:
        tts = gTTS(text, lang='zh-tw')
        if filename is None:
            filename = "{}.mp3".format(temp.name)
        tts.save(filename)
        mixer.init()
        mixer.music.load(filename)
        mixer.music.play()
        while mixer.music.get_busy() == True:
            continue
        mixer.quit()
        
def now_weather(stationName):
    headers = { "Authorization": "CWA-1C69FBA0-5A17-44A9-9529-F3D4AB4C7D28" }
    # 語音辨識臺台轉換
    if stationName[0]=="台":
        stationName = stationName.replace("台", "臺")
    params = {
        "limit": 5,
        "offset": 0,
        "format": "JSON",
        "StationId": "",
        "StationName": stationName,
        "WeatherElement": "AirTemperature,RelativeHumidity",
        "GeoInfo": "CountyName"
    }

    url = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0003-001'
    response = json.loads(requests.get(url, headers=headers, params=params).text)
    temperature = response['records']['Station'][0]['WeatherElement']['AirTemperature']
    relativeHumidity = response['records']['Station'][0]['WeatherElement']['RelativeHumidity']
    # upload to server and say
    url = "http://IP_address/main/weather.php"
    data = {
        'stationName': stationName,
        'temperature': temperature,
        'relativeHumidity': relativeHumidity
    } 
    requests.get(url, params=data)
    print(stationName+"，"+"現在氣溫: "+str(temperature)+"度，相對溼度: "+str(relativeHumidity)+"%")
    say(stationName+"，"+"現在氣溫: "+str(temperature)+"度，相對溼度: "+str(relativeHumidity)+"%")

def future_weather(future_time, stationName):
    headers = { "Authorization": "CWA-1C69FBA0-5A17-44A9-9529-F3D4AB4C7D28" }
    # 語音辨識臺台轉換
    if stationName[0]=="台":
        stationName = stationName.replace("台", "臺")
    params = {
        "limit": 10,
        "offset": 0,
        "format": "JSON",
        "locationName": stationName,  # Change to the desired location
        "sort": "time"
    }
    url = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-C0032-001"
    response = json.loads(requests.get(url, headers=headers, params=params).text)
    for weather in response['records']['location'][0]['weatherElement']:
        for start_time in weather['time']:
            if start_time['startTime']==future_time:
                if weather['elementName']=='Wx':
                    wx = start_time['parameter']['parameterName']
                if weather['elementName']=='PoP':
                    pop = start_time['parameter']['parameterName']
                if weather['elementName']=='MinT':
                    minT = start_time['parameter']['parameterName']
                if weather['elementName']=='CI':
                    ci = start_time['parameter']['parameterName']
                if weather['elementName']=='MaxT':
                    maxT = start_time['parameter']['parameterName']
    # upload to server and say
    url = "http://IP_address/main/weather.php"
    data = {
        'stationName': stationName,
        'future_time': future_time,
        'wx': wx,
        'pop': pop,
        'minT': minT,
        'maxT': maxT,
        'ci': ci
    } 
    requests.get(url, params=data)
    print(stationName+"，"+future_time+"，"+wx+"，濕度: "+pop+"，最低溫度: "+minT+"最高溫度: "+maxT+"舒適度: "+ci)
    say(stationName+"，"+future_time+"，"+wx+"，濕度: "+pop+"，最低溫度: "+minT+"最高溫度: "+maxT+"舒適度: "+ci)
    
if __name__ == "__main__":
    now_weather('台北')
    future_weather('2023-12-12 18:00:00', '台北市')

臺北，現在氣溫: 20.7度，相對溼度: 82%
臺北市，2023-12-12 18:00:00，陰時多雲短暫雨，濕度: 30，最低溫度: 19最高溫度: 20舒適度: 稍有寒意
